In [1]:
import os
import subprocess
from pathlib import Path
import numpy as np
import operator
import json
import math
import resource

In [2]:
DIVISIONS = ['div1', 'div2', 'all']

DIV_THRESHOLD = 1650

In [3]:
division = 'all'

In [4]:
time_limit = 5
memory_limit = 256 * 1024 * 1024

In [5]:
all_students = ['94101737', '94101975', '95105657', '95106494', '95108008', '95109242', '95109383', '96101262', '96101913', '96104881', '96105553', '96105575', '96105626', '96105637', '96105672', '96105683', '96105712', '96105807', '96105829', '96105853', '96105864', '96105875', '96105886', '96105948', '96105983', '96106033', '96106077', '96106117', '96109588', '96109599', '96109606', '96109663', '96109714', '96109736', '96109747', '96109758', '96109899', '96109906', '96110067', '96110129', '96110797', '97100581', '97102044', '97102441', '97105696', '97105771', '97105844', '97105855', '97106262', '97106295', '97106419', '97110155', '97110206', '97213093', '98209048']

In [6]:
import shutil

working_directory = os.path.abspath('')
os.chdir(working_directory)

submissions_path = Path(os.path.join(working_directory, 'submissions'))

tests_path = Path(os.path.join(working_directory, 'tests'))

executables_path = os.path.join(working_directory, f'executables-{division}')

rating_file = 'rating.json'

if os.path.exists(executables_path):
    shutil.rmtree(executables_path)

os.mkdir(executables_path)
    
outputs_path = Path(os.path.join(working_directory, f'outputs-{division}'))
    
if os.path.exists(outputs_path):
    shutil.rmtree(outputs_path)    
    
os.mkdir(outputs_path)

In [8]:
students = list(os.walk(submissions_path))[0][1]

if division != 'div1':
    students += ['dummy']

try:
    with open(rating_file, 'r', encoding='utf-8') as fp:
        rate = json.load(fp)

    for student in students:
        if student not in rate:
            print(f'FATAL ERROR: {student} has no rating!')

    for std in rate:
        if std not in students:
            print(f'{std} did not submit at this round.')
            students.append(std)
except FileNotFoundError:
    print('No rating file.')

if division == 'div1':
    students = list(filter(lambda x: rate[x] >= DIV_THRESHOLD, students))
elif division == 'div2':
    students = list(filter(lambda x: rate[x] < DIV_THRESHOLD, students))
            
final_results = {student: {'ranking': None, 'tests': {}, 'final_score': 0, 'prev_rating': 0, 'new_rating': 0} for student in students}

95109383 did not submit at this round.
97106295 did not submit at this round.
96109736 did not submit at this round.
97213093 did not submit at this round.


In [9]:
print(division, 'contest')
print(f'{len(students)} students we have: {students}')

all contest
45 students we have: ['97100581', '97102044', '97106419', '96104881', '96109599', '94101737', '96105875', '97105771', '96105886', '96106033', '96105626', '96110797', '96105829', '96109606', '96109663', '96110129', '97106262', '96105712', '96105575', '97105844', '96105948', '96105983', '96105553', '97102441', '94170733', '96109906', '97105855', '97105696', '98209048', '96105683', '97110206', '95108008', '96101262', '96106077', '96105807', '96110067', '96109588', '97110155', '96109747', '96109714', 'dummy', '95109383', '97106295', '96109736', '97213093']


In [10]:
print(submissions_path)
print(working_directory)
print(tests_path)
print(executables_path)

/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-5/judge/submissions
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-5/judge
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-5/judge/tests
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-5/judge/executables-all


In [11]:
all_tests = list(map(lambda path: os.path.basename(path).split('in')[1].split('.txt')[0], tests_path.rglob('*.txt')))

In [14]:
print('Compiling codes...')

errors = 0

for student in students:
    cpp_files = list(Path(os.path.join(submissions_path, student)).rglob('*.c*'))
    ans = [] 
    for i in cpp_files:
        if i.name.endswith(".cxx") or i.name.endswith(".cpp"):
            ans.append(i)
    cpp_files = ans
    
    if len(cpp_files) != 1:
        print(f'Something wrong with {student}\'s files. Could not find the cpp file')
        final_results[student]['tests'] = {test: {'score': 0, 'cost': None, 'status': 'No Submission'} for test in all_tests}
        errors += 1
        continue

    cpp_file = cpp_files[0]
    output_file = os.path.join(submissions_path, student, 'output-runnable')
    
    process = subprocess.run(['g++', cpp_file, '-std=c++14', '-O2', '-o', output_file], capture_output=True)
    if process.returncode:
        print(f'Could not compile {student}\'s code.')
        print(f'*******************')
        print(process.stderr.decode()[:300])
        print(f'*******************')              
        final_results[student]['tests'] = {test: {'score': 0, 'cost': None, 'status': 'Compiler Error'} for test in all_tests}
        errors += 1
        continue
    
    
    shutil.move(output_file, os.path.join(executables_path, f'{student}.exec'))

print(f'Compilation finished for {len(students)} codes: {errors} error(s).')
print('\n=================================================\n')

Compiling codes...
Something wrong with dummy's files. Could not find the cpp file
Something wrong with 95109383's files. Could not find the cpp file
Something wrong with 97106295's files. Could not find the cpp file
Something wrong with 96109736's files. Could not find the cpp file
Something wrong with 97213093's files. Could not find the cpp file
Compilation finished for 45 codes: 5 error(s).




In [15]:
print('Executing codes...')

def limit_virtual_memory():
    resource.setrlimit(resource.RLIMIT_AS, (memory_limit, memory_limit))


for executable in list(Path(executables_path).rglob('*.exec')):    
    student_id = os.path.basename(executable).split('.exec')[0]
        
    print(f'Running {student_id}\'s codes')        
        
    for test_file in tests_path.rglob('*.txt'):
        test_name = os.path.basename(test_file).split('.txt')[0]
        
        test_num = test_name[2:]
        
        output_dir = os.path.join(outputs_path, test_name)
        os.makedirs(output_dir, exist_ok=True)
        
        output_path = os.path.join(output_dir, f'output-{student_id}.txt')
        
        input_file = open(test_file)
        output_file = open(output_path, 'w')
        
        try:
            subprocess.run([executable], stdin=input_file, stdout=output_file, timeout=time_limit, preexec_fn=limit_virtual_memory, check=True)
            output_file.close()
        except subprocess.TimeoutExpired:
            print(f'Time limit exceeded on test {test_name}')
            final_results[student_id]['tests'][test_num] = {'score': 0, 'cost': None, 'status': 'Time limit'}
            os.remove(output_path)
        except subprocess.CalledProcessError:
            print(f'Runtime error on test {test_name}')
            final_results[student_id]['tests'][test_num] = {'score': 0, 'cost': None, 'status': 'Runtime Error'}
            os.remove(output_path)
        
        input_file.close()
        
print('\n=================================================\n')

Executing codes...
Running 97105696's codes
Running 97110206's codes
Running 96109599's codes
Running 97106262's codes
Running 97105771's codes
Running 96105575's codes
Running 96105626's codes
Time limit exceeded on test input1
Running 96101262's codes
Running 97110155's codes
Running 96105807's codes
Running 97105855's codes
Running 96109747's codes
Running 97106419's codes
Running 98209048's codes
Time limit exceeded on test input18
Time limit exceeded on test input26
Time limit exceeded on test input22
Time limit exceeded on test input14
Running 96105683's codes
Running 96105553's codes
Running 97105844's codes
Running 94101737's codes
Running 96109663's codes
Running 96105712's codes
Running 97100581's codes
Running 96109588's codes
Running 97102441's codes
Runtime error on test input1
Runtime error on test input25
Runtime error on test input19
Runtime error on test input18
Runtime error on test input24
Runtime error on test input30
Runtime error on test input2
Runtime error on te

In [16]:
checker_files = list(Path(working_directory).rglob('checker.cpp')) 
if len(checker_files) != 1:
    print("Please provide one checker.cpp file")

judge = os.path.join(working_directory, 'judge.exec')    

process = subprocess.run(['g++', checker_files[0], '-std=c++14', '-O2', '-o', judge], capture_output=True)
if process.returncode:
    print(f'Could not compile {student}\'s code:')
    print(process.stderr.decode()[:300])

In [17]:
cost = {}
for s in students:
    cost[s] = {}

for test in list(Path(outputs_path).rglob('in*')):
    this_test = os.path.basename(test) 
    test_num = this_test[2:]
    
    input_file = os.path.join(tests_path, this_test + '.txt')
    
    valid_submissions = list(test.rglob('output*.txt'))
    print("Seeing submissions output for test ", test_num, "\n")

    for student_output in valid_submissions:
        student_id = student_output.name.split('output-')[1].split('.txt')[0]
        
        try:
            out_path = os.path.join(test, f'checker-{student_id}.txt')
            log_path = os.path.join(test, f'log-{student_id}.txt')
            out_file = open(out_path, 'w')
            log_file = open(log_path, 'w')
            
            subprocess.run([judge, input_file, student_output], timeout=2, stdout=out_file, stderr=log_file, check=True)
            
            with open(out_path, "r") as f:
                checker_out = f.read()
            
            out_file.close()
            log_file.close()
            
            cost[student_id][test_num] = int(checker_out)
            print(f"{student_id}'s cost is {checker_out}")
        except subprocess.TimeoutExpired:
            print(f'Time limit exceeded on checker. its not normal')
            os.remove(log_file)
        except subprocess.CalledProcessError:
            print(f'Runtime error on checker. its not normal')
            os.remove(log_file)
        
        

Seeing submissions output for test  put29 

96105683's cost is 70222244847693

96110067's cost is 20227519142829

97106262's cost is 13731495016590

97110206's cost is 24865035149672

96109906's cost is 91764721484507

96105875's cost is 7945526771761

96110129's cost is 136110994655712

95108008's cost is 5030876578244

96105807's cost is 136141772888739

96106077's cost is 261665558451581

97105696's cost is 68419696821358

96105626's cost is 252882517023297

96105829's cost is 17215473383585

96109747's cost is 134748208450845

96105553's cost is 69784500771392

96104881's cost is 6512264355472

96110797's cost is 7743974430909

96109588's cost is 103397620664625

97106419's cost is 6605707157139

97105844's cost is 4977693756956

97102044's cost is 17215473383585

98209048's cost is -1

96101262's cost is 40319805727819

96105575's cost is 60536953831500

96105983's cost is 136141772888739

96109606's cost is 5435260672541

96109599's cost is 45673006788550

97105855's cost is 8044

96104881's cost is 6829719278034

96110797's cost is 5904492085446

96109588's cost is 24481410677791

97106419's cost is 9226257626795

97105844's cost is 4822382415812

97102044's cost is 10605480004931

98209048's cost is -1

96101262's cost is 19265777337981

96105575's cost is 18178705205159

96105983's cost is 87733566719917

96109606's cost is 8090273229952

96109599's cost is 14828717250287

97105855's cost is 6372292583940

96105886's cost is 36578165637155

96109663's cost is 13950965311029

96109714's cost is 18916693881415

94101737's cost is 45886306662854

96106033's cost is 16901308615289

97110155's cost is 130870355438983

97100581's cost is 5274782761619

96105712's cost is 19761857513298

97105771's cost is 6354720305180

Seeing submissions output for test  put19 

96105683's cost is 342981696745

96110067's cost is 155728522005

97106262's cost is 149958324459

97110206's cost is 196627412319

96109906's cost is 222992207128

96105875's cost is 168061524872

9611012

96109663's cost is 84567960272

96109714's cost is 117134444054

94101737's cost is 126327103801

96106033's cost is 102540867261

97110155's cost is 270151298442

97100581's cost is 69677770466

96105712's cost is 112564166285

97105771's cost is 92883965211

Seeing submissions output for test  put4 

96105683's cost is 27639237034239

96110067's cost is 2927281478354

97106262's cost is 4485889764539

97110206's cost is 11666213924979

96109906's cost is 9437178313608

96105875's cost is 4289493018881

96110129's cost is 52565015887538

95108008's cost is 2055302528366

96105807's cost is 53569344572214

96106077's cost is 70131149078510

97105696's cost is 27855092884075

96105626's cost is 46060319548120

96105829's cost is 3148602967447

96109747's cost is 52020215735978

96105553's cost is 32329335989781

96104881's cost is -1

96110797's cost is 3006977889077

96109588's cost is 5273733901589

97106419's cost is 4077462355643

97105844's cost is 1882336784800

97102044's cost is

96105829's cost is 939214678958

96105948's cost is 3473273181419

96109747's cost is 13818002851154

96105553's cost is 7625913510430

96104881's cost is 2437694496591

96110797's cost is 1310487743588

96109588's cost is 1683316562935

97106419's cost is 1521995879953

97105844's cost is 815959481429

97102044's cost is 939214678958

98209048's cost is 17334257092249

96101262's cost is 1292988883001

96105575's cost is 1235990227009

96105983's cost is 14529370182953

96109606's cost is 1640681697551

96109599's cost is 1139377535790

97105855's cost is 1502118344749

96105886's cost is 5302271767506

96109663's cost is 2037396941177

96109714's cost is 1490992401542

94101737's cost is 7687692617761

96106033's cost is 1145091045291

97110155's cost is 21529751123099

97100581's cost is 1184538854804

96105712's cost is 1469270044700

97105771's cost is 1547623533195

Seeing submissions output for test  put24 

96105683's cost is 7209888323241

96110067's cost is 2429174658798

971

97110155's cost is 2025654652778

97100581's cost is 261818239566

96105712's cost is 790249646793

97105771's cost is 326319755804

Seeing submissions output for test  put9 

96105683's cost is 2686500453389

96110067's cost is 545703153185

97106262's cost is 856885302533

97110206's cost is 971504489389

96109906's cost is 1278429644366

96105875's cost is 902116174184

96110129's cost is 4672282032249

95108008's cost is 452766494361

96105807's cost is 4783664960070

96106077's cost is 3983792833884

97105696's cost is 2429508193620

96105626's cost is 3440489745879

96105829's cost is 547837222499

96109747's cost is 4542646842774

96105553's cost is 2724867599531

96104881's cost is 791399909419

96110797's cost is 640926282624

96109588's cost is 1096196242842

97106419's cost is 944435267206

97105844's cost is 444883515332

97102044's cost is 547837222499

98209048's cost is 4033063369165

96101262's cost is 827449562735

96105575's cost is 739129719494

96105983's cost is 47

In [18]:
def prob(ri, rj):
    delta = (rj - ri) / 400.
    return 1.0 / (1.0 + 10**(delta))

def return_student_ranking(all_scores, all_tests, test_weights, use_w = False):
    total_score = {student: 0 for student in students}
    for t in all_tests:
        results = []
        for s in students:
            if not t in all_scores[s]:
                continue
            res = all_scores[s][t]
            if res != -1:
                results.append((res, s))
        results.sort()
        first_three = results
        if len(results) > 3:
            first_three = first_three[:3]
        C = []
        for i in range(len(first_three)):
            C.append(first_three[i][0])
        Cm = np.mean(C)
        for s in students:
            if not t in all_scores[s]:
                continue
            res = all_scores[s][t]
            if res == -1:
                final_results[s]['tests'][t] = {'score': 0, 'cost': None, 'status': 'Invalid'}
                continue
            X = test_weights[t] * min(1, (Cm/res)**(1/3))
            final_results[s]['tests'][t] = {'score': round(X, 3), 'cost': res, 'status': 'Success'}
            total_score[s] += X
    sorted_d = sorted(total_score.items(), key=operator.itemgetter(1), reverse=True)
    rank = []
    eps = 1e-6
    for i, (student, score) in enumerate(sorted_d):
        cnt = 1
        for i2, (student2, score2) in enumerate(sorted_d):
            if score2 - eps > score:
                cnt += 1
        pair_rank = (student, cnt)
        rank.append(pair_rank)
        
        final_results[student]['final_score'] = round(score, 3)
        final_results[student]['ranking'] = cnt
    return rank

def get_new_ratings(rate, standings): # current rates, standing of student ids in latest round
    seed = {}
    for i in rate:
        seed[i] = 1
        for j in rate:
            if i == j:
                continue
            seed[i] += prob(rate[j], rate[i])
    new_rates = {}
    for idx, rank in enumerate(standings): # u: user
        lo, hi = 10, 4000
        u = rank[0]
        place = rank[1]
        m_i = math.sqrt((place) * seed[u])
        while(hi - lo > 1):
            mid = (hi + lo)//2
            new_seed = 1
            for j in rate:
                if j == u:
                    continue
                new_seed += prob(rate[j], mid)
            
            if new_seed < m_i: # over valued rating
                hi = mid
            else:
                lo = mid
        R = (hi + lo) * 0.5
        d_i = (R - rate[u]) * 0.5
        new_rates[u] = int(rate[u] + d_i)
    return new_rates


def update_ratings():
    rating_files = list(Path(working_directory).rglob(rating_file))
    rate = {}
    if len(rating_files) == 0:
        print("NO ratings will set every one to 1500")
        for s in students:
            rate[s] = 1500
    else:
        with open(rating_file, 'r', encoding='utf-8') as fp:
            rate = json.load(fp)
        rate = {key: value for key, value in rate.items() if key in final_results}
    print("previous rates -----")
    print(rate)
    for x, y in rate.items():
        final_results[x]['prev_rating'] = y
    standings = return_student_ranking(cost, all_tests, {t: 100 for t in all_tests}, False)
    print(standings)
    new_rates = get_new_ratings(rate, standings)
    destination = f"rating-{division}.json"
    print("new rates -----")
    print(new_rates)
    for x, y in new_rates.items():
        final_results[x]['new_rating'] = y
    with open(destination, 'w', encoding='utf-8') as fp:
        json.dump(new_rates, fp)

In [19]:
update_ratings()

previous rates -----
{'97105844': 1983, '97105855': 1950, '95108008': 1927, '96109599': 1779, '96105829': 1789, '96110067': 1755, '96109747': 1785, '97100581': 1762, '96109663': 1691, '97110206': 1658, '96105886': 1658, '94101737': 1667, '96109714': 1589, '97102044': 1781, '96106033': 1671, '96106077': 1735, '97105771': 1665, '96104881': 1568, '96105875': 1581, '96105948': 1662, '96109588': 1700, '96109606': 1633, '96105575': 1495, '98209048': 1537, '96101262': 1623, '97102441': 1586, '96105553': 1492, '97105696': 1584, '96105683': 1474, '96105626': 1583, '96110797': 1526, '94170733': 1439, '96110129': 1464, '96105712': 1527, '96105983': 1466, '96105807': 1445, '97110155': 1391, '97106419': 1578, '97106262': 1438, '95109383': 1369, 'dummy': 1349, '96109906': 1550, '97106295': 1389, '96109736': 1419, '97213093': 1352}
[('97105844', 1), ('95108008', 2), ('97100581', 3), ('96110797', 4), ('97105855', 5), ('97105771', 6), ('97102044', 7), ('96105829', 7), ('96110067', 9), ('96105875', 10),

In [20]:
data = sorted(final_results.items(), key= lambda item: item[1]['ranking'])

In [21]:
import xlsxwriter


def export_excel(data):
    workbook = xlsxwriter.Workbook(f'results-{division}.xlsx')
    worksheet = workbook.add_worksheet()

    col = 0
    worksheet.write(0, 0, "Ranking")
    worksheet.write(0, 1, "Student ID")
    worksheet.write(0, 2, "Final score")
    worksheet.write(0, 3, "Prev rating")
    worksheet.write(0, 4, "Rating change")
    worksheet.write(0, 5, "New rating")
    worksheet.write(0, 6, "")

    for i in range(1, len(all_tests) + 1):
        worksheet.write(0, i + 6, f"Test {i} score\n(cost)")
        
    for i in range(len(data)):
        col = 0
        worksheet.write(2*i+1, col, data[i][1]["ranking"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][0])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["final_score"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["prev_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["new_rating"] - data[i][1]["prev_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["new_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, "")
        worksheet.write(2*i+2, col, "")
        col += 1
        for num in sorted(map(lambda x: x[3:], data[i][1]["tests"]), key=int):
            worksheet.write(2 * i + 1, col, data[i][1]["tests"][f"put{num}"]["score"])
            if data[i][1]["tests"][f"put{num}"]['status'] == "Success":
                worksheet.write(2 * i + 2, col, f'({data[i][1]["tests"][f"put{num}"]["cost"]})')
            else:
                worksheet.write(2 * i + 2, col, f'({data[i][1]["tests"][f"put{num}"]["status"]})')
        
            col += 1

    workbook.close()

In [22]:
export_excel(data)

In [23]:
os.remove(judge)

In [52]:
rating_files = [
    '../../Round-2/judge/rating.json', 
    '../../Round-3/judge/rating.json', 
    '../../Round-4/judge/rating.json', 
    '../../Round-5/judge/rating.json',
    '../../Round-5/judge/rating-all.json'
]

ratings = []
for rating_file in rating_files:
    with open(rating_file) as f:
        ratings.append(json.loads(f.read()))
        
        
final_standings = list(zip(*sorted(ratings[-1].items(), key=lambda x: x[1], reverse=True)))[0]


workbook = xlsxwriter.Workbook(f'final-results.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write(0, 0, "Ranking")
worksheet.write(0, 1, "Student ID")



for i in range(len(ratings)):
    worksheet.write(0, i * 3 + 2, f"Round {i + 1} Rating")
    worksheet.write(0, i * 3 + 3, f"Changes")
    worksheet.write(0, i * 3 + 4, f"Ranking")


for i, student in enumerate(final_standings):
    worksheet.write(i + 1, 0, i + 1)
    worksheet.write(i + 1, 1, student)

    for j in range(len(ratings)):
        worksheet.write(i + 1, j * 3 + 2, ratings[j][student])
        worksheet.write(i + 1, j * 3 + 3, ratings[j][student] - (ratings[j-1][student] if j else 1500)) 
        worksheet.write(i + 1, j * 3 + 4, len(list(filter(lambda x:x[1] > ratings[j][student], ratings[j].items()))) + 1)

        
workbook.close()